<a href="https://colab.research.google.com/github/shruthimohan03/Text-to-Video-GenAI/blob/main/Capstone_CNN_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the images and preprocess

In [101]:
import numpy as np
from PIL import Image
import tensorflow as tf

def load_and_preprocess_image(image_path, target_size=(224, 224)):
    # Load the image
    image = Image.open(image_path)

    # Resize the image
    image = image.resize(target_size)

    # Normalization
    image_array = np.array(image) / 255.0

    # Add an extra dimension to match the input shape for the model (batch size, height, width, channels)
    image_array = np.expand_dims(image_array, axis=0)

    return image_array

# Assume image1 and image2 are file paths to your images
image1_path = '/content/image1.jpg'
image2_path = '/content/image2.jpg'

# Preprocess images
image1 = load_and_preprocess_image(image1_path)
image2 = load_and_preprocess_image(image2_path)

CNN model

In [84]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape=(224, 224, 3)):
    model = models.Sequential()

    # First Convolutional Block
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    # Second Convolutional Block
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Third Convolutional Block
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Fourth Convolutional Block
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output and add Dense layers for feature extraction
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(256, activation='relu'))

    # Output layer (optional, can be removed if using for feature extraction only)
    model.add(layers.Dense(128, activation='relu'))

    return model

# Example usage
cnn_model = create_cnn_model()
cnn_model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_44 (Conv2D)                   │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_44 (MaxPooling2D)      │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_45 (Conv2D)                   │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_45 (MaxPooling2D)      │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_46 (Conv2D)                   │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_46 (MaxPooling2D)      │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_47 (Conv2D)                   │ (None, 24, 24, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_47 (MaxPooling2D)      │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 36864)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 512)                 │      18,874,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 128)                 │          32,896 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,427,520 (74.11 MB)

 Trainable params: 19,427,520 (74.11 MB)

 Non-trainable params: 0 (0.00 B)

In [102]:
# Create the CNN model (use the earlier model definition)
cnn_model = create_cnn_model()

# Extract features from the CNN
feature1 = cnn_model.predict(image1)
feature2 = cnn_model.predict(image2)

# Print the shape of the features to understand what the model extracted
print("Extracted feature1 shape:", feature1.shape)
print("Extracted feature2 shape:", feature2.shape)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Extracted feature1 shape: (1, 128)
Extracted feature2 shape: (1, 128)


In [103]:
from sklearn.metrics.pairwise import cosine_similarity
# Calculate Cosine Similarity
cos_sim = cosine_similarity(feature1, feature2)

print("Cosine Similarity:", cos_sim[0][0])

Cosine Similarity: 0.9485668


In [104]:
from scipy.spatial.distance import euclidean

# Calculate Euclidean Distance
eucl_dist = euclidean(feature1.flatten(), feature2.flatten())

print("Euclidean Distance:", eucl_dist)

Euclidean Distance: 0.09113664925098419


Pretrained model: ResNet50

In [55]:
from tensorflow.keras.applications import ResNet50

# Load Pretrained ResNet50 Model without the top fully-connected layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features from the pretrained model
features_image1 = base_model.predict(image1)
features_image2 = base_model.predict(image2)

# Flatten the feature maps (optional, depends on how you want to compare)
features_image1 = features_image1.flatten()
features_image2 = features_image2.flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


In [56]:
# Cosine Similarity
cos_sim = cosine_similarity([features_image1], [features_image2]  )
print("Cosine Similarity with ResNet50:", cos_sim[0][0])

Cosine Similarity with ResNet50: 0.9928214


In [57]:
from scipy.spatial.distance import euclidean

# Calculate Euclidean Distance
eucl_dist = euclidean(features_image1.flatten(), features_image2.flatten())

print("Euclidean Distance:", eucl_dist)

Euclidean Distance: 30.9340877532959


Different images

In [58]:
cat1_path = '/content/cat1.jpg'
cat2_path = '/content/cat2.jpg'

# Preprocess images
cat1 = load_and_preprocess_image(cat1_path)
cat2 = load_and_preprocess_image(cat2_path)

In [59]:
# Extract features from the CNN
feature1_cat = cnn_model.predict(cat1)
feature2_cat = cnn_model.predict(cat2)

# Print the shape of the features to understand what the model extracted
print("Extracted feature1 shape:", feature1_cat.shape)
print("Extracted feature2 shape:", feature2_cat.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Extracted feature1 shape: (1, 128)
Extracted feature2 shape: (1, 128)


In [60]:
# Calculate Cosine Similarity
cos_sim = cosine_similarity(feature1_cat, feature2_cat)

print("Cosine Similarity:", cos_sim[0][0])

Cosine Similarity: 0.95180315


In [70]:
from scipy.spatial.distance import euclidean

# Calculate Euclidean Distance
eucl_dist = euclidean(feature1_cat.flatten(), feature2_cat.flatten())

print("Euclidean Distance:", eucl_dist)

Euclidean Distance: 0.12907668948173523


Dissimilar images

CNN model

In [105]:
dis1_path = '/content/dissimilar1.jpg'
dis2_path = '/content/dissimilar3.jpg'

# Preprocess images
dis1 = load_and_preprocess_image(dis1_path)
dis2 = load_and_preprocess_image(dis2_path)

In [106]:
# Extract features from the CNN
feature1_dis = cnn_model.predict(dis1)
feature2_dis = cnn_model.predict(dis2)

# Print the shape of the features to understand what the model extracted
print("Extracted feature1 shape:", feature1_dis.shape)
print("Extracted feature2 shape:", feature2_dis.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Extracted feature1 shape: (1, 128)
Extracted feature2 shape: (1, 128)


In [107]:
# Calculate Cosine Similarity
cos_sim = cosine_similarity(feature1_dis, feature2_dis)

print("Cosine Similarity:", cos_sim[0][0])

Cosine Similarity: 0.9242361


In [108]:
from scipy.spatial.distance import euclidean

# Calculate Euclidean Distance
eucl_dist = euclidean(feature1_dis.flatten(), feature2_dis.flatten())

print("Euclidean Distance:", eucl_dist)

Euclidean Distance: 0.15794841945171356


resnet

In [47]:
# Extract features from the pretrained model
features_image1_dis = base_model.predict(dis1)
features_image2_dis = base_model.predict(dis2)

# Flatten the feature maps (optional, depends on how you want to compare)
features_image1_dis = features_image1_dis.flatten()
features_image2_dis = features_image2_dis.flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step


In [48]:
# Calculate Cosine Similarity
cos_sim = cosine_similarity([features_image1_dis], [features_image2_dis])

print("Cosine Similarity:", cos_sim[0][0])

Cosine Similarity: 0.9143116


In [49]:
from scipy.spatial.distance import euclidean

# Calculate Euclidean Distance
eucl_dist = euclidean(features_image1_dis.flatten(), features_image2_dis.flatten())

print("Euclidean Distance:", eucl_dist)

Euclidean Distance: 107.73973083496094
